# Simulações de Monte Carlo

Este código é uma implementação do método de Simulação de Monte Carlo Não-Sequencial para a estimação dos índices anuais de confiabilidade:

- LOLP

- LOLE

- EPNS

- EENS

- EIR

### Importar bibliotecas

In [1]:
import numpy as np
import plotly.express as px
import pandas as pd
import openpyxl

### Entrada de dados

A leitura dos dados das usinas é feita a partir do arquivo "Gerac.xlsx".

In [2]:
data = pd.read_excel('Gerac.xlsx', usecols=[0, 1, 2, 4])

# Dados das usinas
DUSI = np.array(data)

# data

A leitura dos dados da curva de carga é feita a partir do arquivo "dataframe_curva_de_carga.xlsx". 

Este arquivo é gerado a partir do arquivo "ler_curva.ipynb", que, para isso, lê os dados do arquivo "curva de carga.xlsx".

In [3]:
# Carga a ser atendida
PLOAD = 2850  # MW


dados_carga = pd.read_excel('dataframe_curva_de_carga.xlsx')

# Dados da carga
DLOAD = np.array(dados_carga)

# dados_carga

### Interpretar dados de entrada 

##### Usinas

In [4]:
usinas = DUSI[:, 0]
n_unidades = DUSI[:, 1]
FOR = DUSI[:, 2] / 100
pot = DUSI[:, 3]

n_usinas = len(usinas)
capacidade_usinas = n_unidades * pot
capacidade_total = sum(capacidade_usinas)  # total do sistema

# print('Usinas: ', usinas)
# print('Número de unidades: ', n_unidades)
# print('Potência: ', pot)
# print('FOR: ', FOR)
# print('Capacidade total: ', capacidade_total)
# print('Capacidade usinas: ', capacidade_usinas)

##### Curva de carga

In [5]:

# Sort DLOAD by the fourth column (ascending order)
DLOAD = DLOAD[DLOAD[:, 3].argsort()]

patamares = DLOAD[:, 0]
niveis = DLOAD[:, 1] / 100
p_patamar = DLOAD[:, 2]  # probabilidade de cada patamar
p_patamar_acumulado = DLOAD[:, 3]  # probabilidade acumulada

# número de patamares
n_patamares = len(patamares)

niveis_str = [str(int(100*nivel)) for nivel in niveis]

# print('Patamares de carga: ', patamares)
# print('Níveis de carga: ', niveis)
# print('Prob. patamar: ', p_patamar)
# print('Prob. acumulada: ', p_patamar_acumulado)
# print('Patamares em string: ', niveis_str)


Plotar gráfico com a probabilidade acumulada dos patamares de carga:

In [6]:

df = pd.DataFrame({
    'Patamar': [' '] + niveis_str + ['  '],
    'Probabilidade': [0] + list(p_patamar_acumulado) + [1]
})

fig = px.line(df, x='Patamar', y='Probabilidade', title='Probabilidade por patamar',line_shape='hv')

# formatação do gráfico
altura = 400
fig.update_layout(height=altura, width=altura*1.5)
fig.update_xaxes(tickvals=niveis_str)
fig.update_yaxes(tickvals=np.arange(0, 1.1, 0.1))
fig.show()

### **Algoritmo Principal**

Definir tolerância de convergência e os números máximo e mínimo de iterações.

In [7]:
tol = 0.005
NSmax = 1e7
NSmin = 100

Execução das simulações de Monte Carlo.

In [8]:

beta_LOLP = tol + 1
NS = 0
soma_LOLP = 0
soma_quad_LOLP = 0
soma_EPNS = 0

while beta_LOLP > tol and NS < NSmax:
    NS += 1
    capacidade = capacidade_total

    U_carga = np.random.rand()

    carga = PLOAD
    for patamar in range(n_patamares):
        if U_carga < p_patamar_acumulado[patamar]:
            carga = PLOAD * niveis[patamar]
            break


    for usi in range(n_usinas):
        for unid in range(n_unidades[usi]):
            Ui = np.random.rand()
            if Ui < FOR[usi]:
                capacidade -= pot[usi]
                
    if capacidade < carga:
        # LOLP
        soma_LOLP += 1
        soma_quad_LOLP += 1 ** 2  # p/ convergencia

        # EPNS
        soma_EPNS += (carga - capacidade)

    v_esperado_LOLP = soma_LOLP / NS  # --> RESPOSTA
    v_esperado_EPNS = soma_EPNS / NS

    # convergencia
    if NS > NSmin:
        variancia_LOLP = (soma_quad_LOLP - NS * v_esperado_LOLP**2) / (NS - 1)
        variancia_do_valor_esperado_LOLP = variancia_LOLP / NS
        beta_LOLP = np.sqrt(variancia_do_valor_esperado_LOLP) / v_esperado_LOLP
        if np.isnan(beta_LOLP):
            beta_LOLP = np.inf


### Resultados 

Obtenção dos valores finais dos índices:


In [9]:
precisao = 8

LOLP = v_esperado_LOLP
EPNS = v_esperado_EPNS
LOLE = LOLP * 8760
EENS = EPNS * 8760


E_sob_curva = sum(100 * niveis * p_patamar * 8760) * carga

EIR = 1 - EENS/E_sob_curva

#### Exibição dos Índices

In [10]:
print("LOLP: ", round(LOLP, precisao),
    "\nEPNS: ", round(EPNS, precisao), "\t MW",
    "\nLOLE: ", round(LOLE, precisao), "\t h/ano",
    "\nEENS: ", round(EENS, precisao), "\t MWh/ano",
    "\nEIR : ", round(EIR , precisao))

LOLP:  0.0232535 
EPNS:  3.60667983 	 MW 
LOLE:  203.70068154 	 h/ano 
EENS:  31594.51532785 	 MWh/ano 
EIR :  0.9999847


#### Informações da Convergência

Número de iterações e Beta_LOLP final.

In [11]:
print('NS =', NS, 
'\nbeta_LOLP =', round(beta_LOLP, precisao))

NS = 1680177 
beta_LOLP = 0.00499999


In [12]:
# criar um dataframe com os resultados, no qual os indices são os resultados e as colunas são os valores

resultado = pd.DataFrame([LOLP, EPNS, LOLE, EENS, EIR], index=['LOLP', 'EPNS (MW)', 'LOLE (h/ano)', 'EENS (MWh/ano)', 'EIR'], columns=['Valor'])

# alinhar à esquerda
resultado = resultado.style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

resultado

,Valor
LOLP,0.023254
EPNS (MW),3.606680
LOLE (h/ano),203.700682
EENS (MWh/ano),31594.515328
EIR,0.999985


In [13]:
# apagar a planilha "SMC" do arquivo "Resultados.xlsx"
wb = openpyxl.load_workbook('Resultados.xlsx')
if 'SMC' in wb.sheetnames:
    wb.remove(wb['SMC'])
    wb.save('Resultados.xlsx')
wb.close()

# escrever o resultado na planilha "SMC" do arquivo "Resultados.xlsx", sem apagar o que já está escrito
writer = pd.ExcelWriter('Resultados.xlsx', engine='openpyxl', mode='a')
resultado.to_excel(writer, sheet_name='SMC')
writer.close()